# Imports

In [ ]:
!pip install xarray pandas numpy cartopy pooch matplotlib

In [ ]:
#Import packaages: 
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

# Load tutorial dataset (in.built in xarray)

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")

# Print the result
## Similar to ncdump -h <filename.nc>

In [ ]:
print(ds)

# Define a point 
## Either point selection OR aggregation

In [ ]:
agg = True

if agg:  
    point = ds.air.mean(dim=["lon","lat"])
    title_str=("Spatial mean over lon lat")
else:
    point = ds.air.isel(lon=40).sel( lat=70, method="nearest") #index + label selection. Latter needs method
    

# Prepare the figure

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 5))  # returns Figure and Axes


#Actuall xarray command to plot a figure: 1 Line! 
point.groupby("time.dayofyear").mean().plot(label="mean", ax=ax1)


#Annoation
plt.legend()
plt.grid()
plt.xlim(0,365)
plt.title(f"Dailymean for dataset over {len(point)} samples\nfrom {point.time.dt.year.min().values} to {point.time.dt.year.max().values}")

# Let us try something a bit more advanced
### Create a list of aggregation methods (min, mean and max) and, using a for loop, apply each aggregation function repectively. Finally, plot all 3 curves on the same figure. 

### Plot should be duly annotated

In [ ]:

aggr_method = "dayofyear"
filtering = True


fig, ax = plt.subplots(figsize=(12, 5))  # returns Figure and Axes
if filtering: 
    fig2, ax2 = plt.subplots(figsize=(12, 5))  # returns Figure and Axes

for name, func in {"max": np.max, "mean": np.mean, "min": np.min}.items(): 
   clim = point.groupby(f"time.{aggr_method}").reduce(func)
   clim.plot(label=f"{name}", ax=ax)
   
    
   if aggr_method == 'dayofyear' and filtering : 
       print(f"\t[+] {name} for day of the year - using rolling mean to filter data")
       window = 10
       clim.rolling(dayofyear=window, center=True).mean().plot(lw=2.5, ls='-.', color='black',alpha=.5, ax=ax)
       clim.rolling(dayofyear=window, center=True).mean().plot(lw=2.5,  ax=ax2, label=f"{name}")
       #print(clim.rolling(center=True, dayofyear=1).mean())
      
    
# Update title once, after the loop
if aggr_method == "dayofyear" and filtering:
    title_str += f"\n - added rolling mean with {window}-day window"
    
if agg: 
    ax.set_title(title_str)
    ax2.set_title(f"Only filtered climatology - rolling windows of {window} days")
    ax2.legend()
#plt.xlim(0, 365)
ax.legend()
plt.grid()